In [ ]:
!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 7.6 MB/s eta 0:00:00


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import sklearn
import pandas as pd
from google.colab import drive
from catboost import CatBoostClassifier, Pool
from sklearn.model_selection import train_test_split

In [ ]:
drive.mount("/content/drive")


Mounted at /content/drive


In [ ]:
!unzip /content/drive/MyDrive/credit_scoring_data/home-credit-default-risk.zip -d /content/data

Archive:  /content/drive/MyDrive/credit_scoring_data/home-credit-default-risk.zip
  inflating: /content/data/HomeCredit_columns_description.csv  
  inflating: /content/data/POS_CASH_balance.csv  
  inflating: /content/data/application_test.csv  
  inflating: /content/data/application_train.csv  
  inflating: /content/data/bureau.csv  
  inflating: /content/data/bureau_balance.csv  
  inflating: /content/data/credit_card_balance.csv  
  inflating: /content/data/installments_payments.csv  
  inflating: /content/data/previous_application.csv  
  inflating: /content/data/sample_submission.csv  


# EDA

In [ ]:
app = pd.read_csv('/content/data/application_train.csv')
bureau = pd.read_csv('/content/data/bureau.csv')
bur_balance = pd.read_csv('/content/data/bureau_balance.csv')
credit_card_balance = pd.read_csv('/content/data/credit_card_balance.csv')
installments_payments = pd.read_csv('/content/data/installments_payments.csv')
prev_app = pd.read_csv('/content/data/previous_application.csv')
pos_cash_balance = pd.read_csv('/content/data/POS_CASH_balance.csv')



In [ ]:
app.columns.tolist()

['SK_ID_CURR',
 'TARGET',
 'NAME_CONTRACT_TYPE',
 'CODE_GENDER',
 'FLAG_OWN_CAR',
 'FLAG_OWN_REALTY',
 'CNT_CHILDREN',
 'AMT_INCOME_TOTAL',
 'AMT_CREDIT',
 'AMT_ANNUITY',
 'AMT_GOODS_PRICE',
 'NAME_TYPE_SUITE',
 'NAME_INCOME_TYPE',
 'NAME_EDUCATION_TYPE',
 'NAME_FAMILY_STATUS',
 'NAME_HOUSING_TYPE',
 'REGION_POPULATION_RELATIVE',
 'DAYS_BIRTH',
 'DAYS_EMPLOYED',
 'DAYS_REGISTRATION',
 'DAYS_ID_PUBLISH',
 'OWN_CAR_AGE',
 'FLAG_MOBIL',
 'FLAG_EMP_PHONE',
 'FLAG_WORK_PHONE',
 'FLAG_CONT_MOBILE',
 'FLAG_PHONE',
 'FLAG_EMAIL',
 'OCCUPATION_TYPE',
 'CNT_FAM_MEMBERS',
 'REGION_RATING_CLIENT',
 'REGION_RATING_CLIENT_W_CITY',
 'WEEKDAY_APPR_PROCESS_START',
 'HOUR_APPR_PROCESS_START',
 'REG_REGION_NOT_LIVE_REGION',
 'REG_REGION_NOT_WORK_REGION',
 'LIVE_REGION_NOT_WORK_REGION',
 'REG_CITY_NOT_LIVE_CITY',
 'REG_CITY_NOT_WORK_CITY',
 'LIVE_CITY_NOT_WORK_CITY',
 'ORGANIZATION_TYPE',
 'EXT_SOURCE_1',
 'EXT_SOURCE_2',
 'EXT_SOURCE_3',
 'APARTMENTS_AVG',
 'BASEMENTAREA_AVG',
 'YEARS_BEGINEXPLUATATION_A

In [ ]:
bureau.columns.tolist()

['SK_ID_CURR',
 'SK_ID_BUREAU',
 'CREDIT_ACTIVE',
 'CREDIT_CURRENCY',
 'DAYS_CREDIT',
 'CREDIT_DAY_OVERDUE',
 'DAYS_CREDIT_ENDDATE',
 'DAYS_ENDDATE_FACT',
 'AMT_CREDIT_MAX_OVERDUE',
 'CNT_CREDIT_PROLONG',
 'AMT_CREDIT_SUM',
 'AMT_CREDIT_SUM_DEBT',
 'AMT_CREDIT_SUM_LIMIT',
 'AMT_CREDIT_SUM_OVERDUE',
 'CREDIT_TYPE',
 'DAYS_CREDIT_UPDATE',
 'AMT_ANNUITY']

In [ ]:
na_cnt = app.isna().sum()
na_pct = na_cnt / len(app) * 100
miss = pd.DataFrame({
    'na_count': na_cnt,
    'na_pct': na_pct
}).sort_values('na_pct', ascending=False)
miss

,na_count,na_pct
COMMONAREA_AVG,214865,69.872297
COMMONAREA_MODE,214865,69.872297
COMMONAREA_MEDI,214865,69.872297
NONLIVINGAPARTMENTS_MEDI,213514,69.432963
NONLIVINGAPARTMENTS_MODE,213514,69.432963
...,...,...
FLAG_DOCUMENT_16,0,0.000000
FLAG_DOCUMENT_15,0,0.000000
FLAG_DOCUMENT_14,0,0.000000
FLAG_DOCUMENT_20,0,0.000000


In [ ]:
train_ids = app['SK_ID_CURR']

# список колонок на удаление
drop_cols = [

    # ID
    'SK_ID_CURR',

    # COMMONAREA
    'COMMONAREA_AVG', 'COMMONAREA_MODE', 'COMMONAREA_MEDI',

    # NONLIVING APARTMENTS
    'NONLIVINGAPARTMENTS_AVG',
    'NONLIVINGAPARTMENTS_MODE',
    'NONLIVINGAPARTMENTS_MEDI',

    # NONLIVING AREA
    'NONLIVINGAREA_AVG',
    'NONLIVINGAREA_MODE',
    'NONLIVINGAREA_MEDI',

    # LAND AREA
    'LANDAREA_AVG',
    'LANDAREA_MODE',
    'LANDAREA_MEDI',

    # BASEMENT
    'BASEMENTAREA_AVG',
    'BASEMENTAREA_MODE',
    'BASEMENTAREA_MEDI',

    # LIVING APARTMENTS
    'LIVINGAPARTMENTS_AVG',
    'LIVINGAPARTMENTS_MODE',
    'LIVINGAPARTMENTS_MEDI',

    # FLOORS MIN
    'FLOORSMIN_AVG',
    'FLOORSMIN_MODE',
    'FLOORSMIN_MEDI',

    # FLOORS MAX (убираем по твоему решению)
    'FLOORSMAX_AVG',
    'FLOORSMAX_MODE',
    'FLOORSMAX_MEDI',

    # ENTRANCES
    'ENTRANCES_AVG',
    'ENTRANCES_MODE',
    'ENTRANCES_MEDI',

    # YEARS
    'YEARS_BUILD_AVG',
    'YEARS_BUILD_MODE',
    'YEARS_BUILD_MEDI',

    'YEARS_BEGINEXPLUATATION_AVG',
    'YEARS_BEGINEXPLUATATION_MODE',
    'YEARS_BEGINEXPLUATATION_MEDI',
    'OWN_CAR_AGE',
    # OTHER HOUSING
    'FONDKAPREMONT_MODE',
    'HOUSETYPE_MODE',
    'WALLSMATERIAL_MODE',
    'EMERGENCYSTATE_MODE',

    # AREA SUMMARY
    'TOTALAREA_MODE',

    # LIFTS
    'ELEVATORS_AVG',
    'ELEVATORS_MODE',
    'ELEVATORS_MEDI'
]

# удаляем (errors='ignore' — чтобы не падало, если колонки нет)
app = app.drop(columns=drop_cols, errors='ignore')
app.columns.tolist()

['TARGET',
 'NAME_CONTRACT_TYPE',
 'CODE_GENDER',
 'FLAG_OWN_CAR',
 'FLAG_OWN_REALTY',
 'AMT_INCOME_TOTAL',
 'AMT_CREDIT',
 'AMT_ANNUITY',
 'NAME_TYPE_SUITE',
 'NAME_INCOME_TYPE',
 'NAME_EDUCATION_TYPE',
 'NAME_FAMILY_STATUS',
 'NAME_HOUSING_TYPE',
 'REGION_POPULATION_RELATIVE',
 'DAYS_BIRTH',
 'DAYS_EMPLOYED',
 'DAYS_REGISTRATION',
 'DAYS_ID_PUBLISH',
 'FLAG_EMP_PHONE',
 'FLAG_WORK_PHONE',
 'FLAG_PHONE',
 'FLAG_EMAIL',
 'OCCUPATION_TYPE',
 'CNT_FAM_MEMBERS',
 'REGION_RATING_CLIENT_W_CITY',
 'WEEKDAY_APPR_PROCESS_START',
 'HOUR_APPR_PROCESS_START',
 'REG_REGION_NOT_LIVE_REGION',
 'REG_REGION_NOT_WORK_REGION',
 'REG_CITY_NOT_LIVE_CITY',
 'REG_CITY_NOT_WORK_CITY',
 'ORGANIZATION_TYPE',
 'EXT_SOURCE_1',
 'EXT_SOURCE_2',
 'EXT_SOURCE_3',
 'APARTMENTS_AVG',
 'OBS_30_CNT_SOCIAL_CIRCLE',
 'DEF_30_CNT_SOCIAL_CIRCLE',
 'DAYS_LAST_PHONE_CHANGE',
 'FLAG_DOCUMENT_3',
 'AMT_REQ_CREDIT_BUREAU_HOUR',
 'AMT_REQ_CREDIT_BUREAU_DAY',
 'AMT_REQ_CREDIT_BUREAU_WEEK',
 'AMT_REQ_CREDIT_BUREAU_MON',
 'AMT_REQ_

In [ ]:
app.columns.tolist()

['TARGET',
 'NAME_CONTRACT_TYPE',
 'CODE_GENDER',
 'FLAG_OWN_CAR',
 'FLAG_OWN_REALTY',
 'CNT_CHILDREN',
 'AMT_INCOME_TOTAL',
 'AMT_CREDIT',
 'AMT_ANNUITY',
 'AMT_GOODS_PRICE',
 'NAME_TYPE_SUITE',
 'NAME_INCOME_TYPE',
 'NAME_EDUCATION_TYPE',
 'NAME_FAMILY_STATUS',
 'NAME_HOUSING_TYPE',
 'REGION_POPULATION_RELATIVE',
 'DAYS_BIRTH',
 'DAYS_EMPLOYED',
 'DAYS_REGISTRATION',
 'DAYS_ID_PUBLISH',
 'OWN_CAR_AGE',
 'FLAG_MOBIL',
 'FLAG_EMP_PHONE',
 'FLAG_WORK_PHONE',
 'FLAG_CONT_MOBILE',
 'FLAG_PHONE',
 'FLAG_EMAIL',
 'OCCUPATION_TYPE',
 'CNT_FAM_MEMBERS',
 'REGION_RATING_CLIENT',
 'REGION_RATING_CLIENT_W_CITY',
 'WEEKDAY_APPR_PROCESS_START',
 'HOUR_APPR_PROCESS_START',
 'REG_REGION_NOT_LIVE_REGION',
 'REG_REGION_NOT_WORK_REGION',
 'LIVE_REGION_NOT_WORK_REGION',
 'REG_CITY_NOT_LIVE_CITY',
 'REG_CITY_NOT_WORK_CITY',
 'LIVE_CITY_NOT_WORK_CITY',
 'ORGANIZATION_TYPE',
 'EXT_SOURCE_1',
 'EXT_SOURCE_2',
 'EXT_SOURCE_3',
 'APARTMENTS_AVG',
 'LIVINGAREA_AVG',
 'APARTMENTS_MODE',
 'LIVINGAREA_MODE',
 'AP

In [ ]:
corr = app.corr(numeric_only=True)
pd.set_option('display.max_rows', None)
display(corr['TARGET'].sort_values(ascending=False))

,TARGET
TARGET,1.000000
DAYS_BIRTH,0.078239
REGION_RATING_CLIENT_W_CITY,0.060893
REGION_RATING_CLIENT,0.058899
DAYS_LAST_PHONE_CHANGE,0.055218
DAYS_ID_PUBLISH,0.051457
REG_CITY_NOT_WORK_CITY,0.050994
FLAG_EMP_PHONE,0.045982
REG_CITY_NOT_LIVE_CITY,0.044395
FLAG_DOCUMENT_3,0.044346


In [ ]:
drop_simple_flags = [
    'FLAG_MOBIL',
    'FLAG_CONT_MOBILE',
]

doc_cols = [c for c in app.columns if c.startswith('FLAG_DOCUMENT_')]

doc_cols_to_drop = [c for c in doc_cols if c != 'FLAG_DOCUMENT_3']

drop_cols = drop_simple_flags + doc_cols_to_drop

app = app.drop(columns=drop_cols, errors='ignore')

In [ ]:
corr_abs = corr.abs()
high_corr = corr_abs.where(
    np.triu(np.ones(corr_abs.shape), k=1).astype(bool)
).stack().sort_values(ascending=False)
high_corr

,,0
DAYS_EMPLOYED,FLAG_EMP_PHONE,0.999755
OBS_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,0.998490
LIVINGAREA_AVG,LIVINGAREA_MEDI,0.995596
APARTMENTS_AVG,APARTMENTS_MEDI,0.995081
AMT_CREDIT,AMT_GOODS_PRICE,0.986968
APARTMENTS_MODE,APARTMENTS_MEDI,0.977193
LIVINGAREA_MODE,LIVINGAREA_MEDI,0.974743
APARTMENTS_AVG,APARTMENTS_MODE,0.973259
LIVINGAREA_AVG,LIVINGAREA_MODE,0.972050
REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,0.950842


In [ ]:
drop_high_corr_columns = ['OBS_60_CNT_SOCIAL_CIRCLE', 'LIVINGAREA_MEDI',
                          'APARTMENTS_MEDI', 'AMT_GOODS_PRICE', 'APARTMENTS_MODE',
                          'LIVINGAREA_MODE', 'REGION_RATING_CLIENT', 'LIVINGAREA_AVG',
                          'CNT_CHILDREN', 'DEF_60_CNT_SOCIAL_CIRCLE', 'LIVE_REGION_NOT_WORK_REGION',
                          'LIVE_CITY_NOT_WORK_CITY']
app = app.drop(columns=drop_high_corr_columns, errors='ignore')
app.columns.tolist()


['TARGET',
 'NAME_CONTRACT_TYPE',
 'CODE_GENDER',
 'FLAG_OWN_CAR',
 'FLAG_OWN_REALTY',
 'AMT_INCOME_TOTAL',
 'AMT_CREDIT',
 'AMT_ANNUITY',
 'NAME_TYPE_SUITE',
 'NAME_INCOME_TYPE',
 'NAME_EDUCATION_TYPE',
 'NAME_FAMILY_STATUS',
 'NAME_HOUSING_TYPE',
 'REGION_POPULATION_RELATIVE',
 'DAYS_BIRTH',
 'DAYS_EMPLOYED',
 'DAYS_REGISTRATION',
 'DAYS_ID_PUBLISH',
 'FLAG_EMP_PHONE',
 'FLAG_WORK_PHONE',
 'FLAG_PHONE',
 'FLAG_EMAIL',
 'OCCUPATION_TYPE',
 'CNT_FAM_MEMBERS',
 'REGION_RATING_CLIENT_W_CITY',
 'WEEKDAY_APPR_PROCESS_START',
 'HOUR_APPR_PROCESS_START',
 'REG_REGION_NOT_LIVE_REGION',
 'REG_REGION_NOT_WORK_REGION',
 'REG_CITY_NOT_LIVE_CITY',
 'REG_CITY_NOT_WORK_CITY',
 'ORGANIZATION_TYPE',
 'EXT_SOURCE_1',
 'EXT_SOURCE_2',
 'EXT_SOURCE_3',
 'APARTMENTS_AVG',
 'OBS_30_CNT_SOCIAL_CIRCLE',
 'DEF_30_CNT_SOCIAL_CIRCLE',
 'DAYS_LAST_PHONE_CHANGE',
 'FLAG_DOCUMENT_3',
 'AMT_REQ_CREDIT_BUREAU_HOUR',
 'AMT_REQ_CREDIT_BUREAU_DAY',
 'AMT_REQ_CREDIT_BUREAU_WEEK',
 'AMT_REQ_CREDIT_BUREAU_MON',
 'AMT_REQ_

In [ ]:
drop_useless_cols = ['AMT_REQ_CREDIT_BUREAU_HOUR', 'AMT_REQ_CREDIT_BUREAU_DAY', 'AMT_REQ_CREDIT_BUREAU_WEEK',
                     'FLAG_EMAIL', 'FLAG_WORK_PHONE', 'FLAG_PHONE', 'HOUR_APPR_PROCESS_START', 'REG_REGION_NOT_LIVE_REGION',
                     'REG_REGION_NOT_WORK_REGION']
app = app.drop(columns=drop_useless_cols, errors='ignore')
app.columns.tolist()
pd.set_option('display.max_columns', None)
app.head(10)

,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,FLAG_EMP_PHONE,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_3,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,1,Cash loans,M,N,Y,202500.0,406597.5,24700.5,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.018801,-9461,-637,-3648.0,-2120,1,Laborers,1.0,2,WEDNESDAY,0,0,Business Entity Type 3,0.083037,0.262949,0.139376,0.0247,2.0,2.0,-1134.0,1,0.0,0.0,1.0
1,0,Cash loans,F,N,N,270000.0,1293502.5,35698.5,Family,State servant,Higher education,Married,House / apartment,0.003541,-16765,-1188,-1186.0,-291,1,Core staff,2.0,1,MONDAY,0,0,School,0.311267,0.622246,NaN,0.0959,1.0,0.0,-828.0,1,0.0,0.0,0.0
2,0,Revolving loans,M,Y,Y,67500.0,135000.0,6750.0,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.010032,-19046,-225,-4260.0,-2531,1,Laborers,1.0,2,MONDAY,0,0,Government,NaN,0.555912,0.729567,NaN,0.0,0.0,-815.0,0,0.0,0.0,0.0
3,0,Cash loans,F,N,Y,135000.0,312682.5,29686.5,Unaccompanied,Working,Secondary / secondary special,Civil marriage,House / apartment,0.008019,-19005,-3039,-9833.0,-2437,1,Laborers,2.0,2,WEDNESDAY,0,0,Business Entity Type 3,NaN,0.650442,NaN,NaN,2.0,0.0,-617.0,1,NaN,NaN,NaN
4,0,Cash loans,M,N,Y,121500.0,513000.0,21865.5,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.028663,-19932,-3038,-4311.0,-3458,1,Core staff,1.0,2,THURSDAY,0,1,Religion,NaN,0.322738,NaN,NaN,0.0,0.0,-1106.0,0,0.0,0.0,0.0
5,0,Cash loans,M,N,Y,99000.0,490495.5,27517.5,"Spouse, partner",State servant,Secondary / secondary special,Married,House / apartment,0.035792,-16941,-1588,-4970.0,-477,1,Laborers,2.0,2,WEDNESDAY,0,0,Other,NaN,0.354225,0.621226,NaN,0.0,0.0,-2536.0,1,0.0,1.0,1.0
6,0,Cash loans,F,Y,Y,171000.0,1560726.0,41301.0,Unaccompanied,Commercial associate,Higher education,Married,House / apartment,0.035792,-13778,-3130,-1213.0,-619,1,Accountants,3.0,2,SUNDAY,0,0,Business Entity Type 3,0.774761,0.724000,0.492060,NaN,1.0,0.0,-1562.0,0,1.0,1.0,2.0
7,0,Cash loans,M,Y,Y,360000.0,1530000.0,42075.0,Unaccompanied,State servant,Higher education,Married,House / apartment,0.003122,-18850,-449,-4597.0,-2379,1,Managers,2.0,3,MONDAY,0,1,Other,NaN,0.714279,0.540654,NaN,2.0,0.0,-1070.0,1,0.0,0.0,0.0
8,0,Cash loans,F,N,Y,112500.0,1019610.0,33826.5,Children,Pensioner,Secondary / secondary special,Married,House / apartment,0.018634,-20099,365243,-7427.0,-3514,0,NaN,2.0,2,WEDNESDAY,0,0,XNA,0.587334,0.205747,0.751724,NaN,1.0,0.0,0.0,1,0.0,0.0,1.0
9,0,Revolving loans,M,N,Y,135000.0,405000.0,20250.0,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.019689,-14469,-2019,-14437.0,-3992,1,Laborers,1.0,2,THURSDAY,0,0,Electricity,NaN,0.746644,NaN,NaN,2.0,0.0,-1673.0,0,NaN,NaN,NaN


In [ ]:
app['DAYS_EMPLOYED'] = app['DAYS_EMPLOYED'].replace(365243, np.nan)
app['CREDIT_TO_INCOME'] = app['AMT_CREDIT'] / app['AMT_INCOME_TOTAL']
app['ANNUITY_TO_INCOME'] = app['AMT_ANNUITY'] / app['AMT_INCOME_TOTAL']
X = app.drop(columns=['TARGET'])
y = app['TARGET']
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
cat_features = ['NAME_CONTRACT_TYPE', 'CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY', 'NAME_TYPE_SUITE',
                'NAME_INCOME_TYPE', 'NAME_EDUCATION_TYPE', 'NAME_FAMILY_STATUS', 'NAME_HOUSING_TYPE',
                'OCCUPATION_TYPE', 'WEEKDAY_APPR_PROCESS_START', 'ORGANIZATION_TYPE']
X_train[cat_features] = X_train[cat_features].fillna('__MISSING__')
X_val[cat_features]   = X_val[cat_features].fillna('__MISSING__')
train_pool = Pool(X_train, y_train, cat_features=cat_features)
val_pool = Pool(X_val, y_val, cat_features=cat_features)
model = CatBoostClassifier(iterations=1000,
                           learning_rate=0.05,
                           max_depth=6,
                           loss_function='Logloss',
                           early_stopping_rounds=200,
                           eval_metric='AUC',
                           verbose=200)
model.fit(train_pool, eval_set=val_pool, use_best_model=True)


0:	test: 0.6282298	best: 0.6282298 (0)	total: 1.42s	remaining: 23m 38s
200:	test: 0.7540062	best: 0.7540062 (200)	total: 2m 18s	remaining: 9m 12s
400:	test: 0.7584861	best: 0.7584861 (400)	total: 4m 39s	remaining: 6m 57s
600:	test: 0.7599458	best: 0.7599777 (593)	total: 7m 3s	remaining: 4m 41s
800:	test: 0.7608577	best: 0.7609097 (766)	total: 9m 31s	remaining: 2m 22s
999:	test: 0.7612954	best: 0.7613254 (992)	total: 11m 51s	remaining: 0us

bestTest = 0.7613254079
bestIteration = 992

Shrink model to first 993 iterations.


In [ ]:
imp = model.get_feature_importance(type="PredictionValuesChange")
feat_imp = pd.Series(data=imp, index=X_train.columns).sort_values(ascending=False)
feat_imp

,0
EXT_SOURCE_3,18.168848
EXT_SOURCE_2,15.050274
EXT_SOURCE_1,6.720582
DAYS_BIRTH,6.092670
AMT_CREDIT,5.180892
DAYS_EMPLOYED,4.458634
AMT_ANNUITY,4.111834
NAME_EDUCATION_TYPE,2.619178
DAYS_REGISTRATION,2.605155
CODE_GENDER,2.475647


In [ ]:
test_app = pd.read_csv('/content/data/application_test.csv')
def transform_app_like_train(df: pd.DataFrame) -> pd.DataFrame:

    df = df.copy()
    DROP_COLS_STATIC = [

        # ID
        'SK_ID_CURR',

        # COMMONAREA
        'COMMONAREA_AVG', 'COMMONAREA_MODE', 'COMMONAREA_MEDI',

        # NONLIVING APARTMENTS
        'NONLIVINGAPARTMENTS_AVG', 'NONLIVINGAPARTMENTS_MODE', 'NONLIVINGAPARTMENTS_MEDI',

        # NONLIVING AREA
        'NONLIVINGAREA_AVG', 'NONLIVINGAREA_MODE', 'NONLIVINGAREA_MEDI',

        # LAND AREA
        'LANDAREA_AVG', 'LANDAREA_MODE', 'LANDAREA_MEDI',

        # BASEMENT
        'BASEMENTAREA_AVG', 'BASEMENTAREA_MODE', 'BASEMENTAREA_MEDI',

        # LIVING APARTMENTS
        'LIVINGAPARTMENTS_AVG', 'LIVINGAPARTMENTS_MODE', 'LIVINGAPARTMENTS_MEDI',

        # FLOORS MIN
        'FLOORSMIN_AVG', 'FLOORSMIN_MODE', 'FLOORSMIN_MEDI',

        # FLOORS MAX
        'FLOORSMAX_AVG', 'FLOORSMAX_MODE', 'FLOORSMAX_MEDI',

        # ENTRANCES
        'ENTRANCES_AVG', 'ENTRANCES_MODE', 'ENTRANCES_MEDI',

        # YEARS
        'YEARS_BUILD_AVG', 'YEARS_BUILD_MODE', 'YEARS_BUILD_MEDI',

        'YEARS_BEGINEXPLUATATION_AVG',
        'YEARS_BEGINEXPLUATATION_MODE',
        'YEARS_BEGINEXPLUATATION_MEDI',

        'OWN_CAR_AGE',

        # HOUSING
        'FONDKAPREMONT_MODE',
        'HOUSETYPE_MODE',
        'WALLSMATERIAL_MODE',
        'EMERGENCYSTATE_MODE',

        # AREA
        'TOTALAREA_MODE',

        # LIFTS
        'ELEVATORS_AVG', 'ELEVATORS_MODE', 'ELEVATORS_MEDI',

        # SIMPLE FLAGS
        'FLAG_MOBIL',
        'FLAG_CONT_MOBILE',

        # HIGH CORR
        'OBS_60_CNT_SOCIAL_CIRCLE',
        'LIVINGAREA_MEDI',
        'APARTMENTS_MEDI',
        'AMT_GOODS_PRICE',
        'APARTMENTS_MODE',
        'LIVINGAREA_MODE',
        'REGION_RATING_CLIENT',
        'LIVINGAREA_AVG',
        'CNT_CHILDREN',
        'DEF_60_CNT_SOCIAL_CIRCLE',
        'LIVE_REGION_NOT_WORK_REGION',
        'LIVE_CITY_NOT_WORK_CITY',

        # USELESS
        'AMT_REQ_CREDIT_BUREAU_HOUR',
        'AMT_REQ_CREDIT_BUREAU_DAY',
        'AMT_REQ_CREDIT_BUREAU_WEEK',
        'FLAG_EMAIL',
        'FLAG_WORK_PHONE',
        'FLAG_PHONE',
        'HOUR_APPR_PROCESS_START',
        'REG_REGION_NOT_LIVE_REGION',
        'REG_REGION_NOT_WORK_REGION',
    ]


    # --- 1. основной drop ---
    df = df.drop(columns=DROP_COLS_STATIC, errors="ignore")

    # --- 2. FLAG_DOCUMENT_* (оставляем только 3) ---
    doc_cols = [c for c in df.columns if c.startswith("FLAG_DOCUMENT_")]
    doc_cols_to_drop = [c for c in doc_cols if c != "FLAG_DOCUMENT_3"]

    df = df.drop(columns=doc_cols_to_drop, errors="ignore")

    # --- 3. feature engineering ---

    if "DAYS_EMPLOYED" in df.columns:
        df["DAYS_EMPLOYED"] = df["DAYS_EMPLOYED"].replace(365243, np.nan)

    if {"AMT_CREDIT", "AMT_INCOME_TOTAL"}.issubset(df.columns):
        denom = df["AMT_INCOME_TOTAL"].replace(0, np.nan)
        df["CREDIT_TO_INCOME"] = df["AMT_CREDIT"] / denom

    if {"AMT_ANNUITY", "AMT_INCOME_TOTAL"}.issubset(df.columns):
        denom = df["AMT_INCOME_TOTAL"].replace(0, np.nan)
        df["ANNUITY_TO_INCOME"] = df["AMT_ANNUITY"] / denom

    # --- 4. убрать TARGET ---
    if "TARGET" in df.columns:
        df = df.drop(columns=["TARGET"])

    return df
test_ID_CURR = test_app['SK_ID_CURR']
X_test = transform_app_like_train(test_app)
X_test[cat_features] = X_test[cat_features].fillna('__MISSING__')
test_pool = Pool(X_test, cat_features=cat_features)
y_test = model.predict_proba(test_pool)[:, 1]
submission = pd.DataFrame({
    "SK_ID_CURR": test_ID_CURR,
    "TARGET": y_test
})
submission.to_csv("submission.csv", index=False)
